In [29]:
import config
import json
import requests
import vulture_scraper as vulture
api_key = config.API_key
import time 
import mysql.connector
from mysql.connector import errorcode
import pandas as pd

In [30]:
## Connect to DB server on AWS
## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = "Movies"       #can say what database we expect everything loaded into
)
cursor = cnx.cursor()

In [31]:
#write a query entry to creat a table
create_query = """
CREATE TABLE tmdb (
      id varchar(50) NOT NULL,
      collection varchar(50),
      budget int,
      genres varchar(50),
      original_language varchar(50),
      overview varchar(50),
      title varchar(50),
      popularity float,
      production_companies varchar(50),
      production_country varchar(50),
      release_date date,
      run_time int,
      spoken_lang varchar(50),
      tagline varchar(50),
      vote_avg float,
      vote_count int,
      imdb_id varchar(50),
      review_count int,
      rating float
    )
"""

In [32]:
#try to create database table
try:
    print("Creating a new table")
    cursor.execute(create_query)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("already exists.")
    else:
        print(err.msg)
else:
    print("OK")
        
cursor.close()
cnx.close()

Creating a new table
already exists.


In [33]:
#write a function that gets a list of dicts, conversts the data to tuples and inserts into our db
def db_insert(movie_dict):
    list_of_tuples = []
    for movie in movie_dict:
        temp_tuple = ()
        temp_tuple = (movie['id'],
              movie['collection'],
              movie['budget'],
              movie['genres'],
              movie['original_language'],
              movie['overview'],
              movie['title'],
              movie['popularity'],
              movie['production_companies'],
              movie['production_country'],
              movie['release_date'],
              movie['run_time'],
              movie['spoken_lang'],
              movie['tagline'],
              movie['vote_avg'],
              movie['vote_count'],
              movie['imdb_id'],
              movie['review_count'],
              movie['rating'])
        list_of_tuples.append(temp_tuple)
    
    
## NOW we insert into out db
    #open connection
    cnx = mysql.connector.connect(
    host = config.host,
    user = config.admin,
    passwd = config.password,
    database = "Movies")
    
    #create cursor
    cursor = cnx.cursor()
    
    
    insert_statement = ("""INSERT INTO tmdb(
                      id,
                      collection,
                      budget,
                      genres,
                      original_language,
                      overview,
                      title,
                      popularity,
                      production_companies ,
                      production_country,
                      release_date,
                      run_time,
                      spoken_lang,
                      tagline,
                      vote_avg,
                      vote_count,
                      imdb_id,
                      review_count,
                      rating float (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,)""")
    
    cursor.executemany(insert_statement, list_of_tuples)
    cnx.commit() 
    
    cursor.close()
    cnx.close()

In [ ]:
## returns data from Movie DB's list of popular movies. updated daily 
def get_popular(api_key):

    url = "https://api.themoviedb.org/3/movie/popular?api_key={}&language=en-US".format(api_key)
    return requests.get(url).json()

In [ ]:
## returns only the titles from Movie DB's list of popular movies 
def get_popular_titles(api_key):
    data = get_popular(api_key)
    return [x['title'] for x in data['results']]

get_popular_titles(api_key)[:5]

In [ ]:
## returns data from Movie DB's list of top rated movies 
def get_top_rated(api_key):
    url = "https://api.themoviedb.org/3/movie/top_rated?api_key={}&language=en-US".format(api_key)
    return requests.get(url).json()

In [ ]:
## returns list of IDs from Movie DB's top rated movies 
def get_top_rated_ids(api_key):
    data = get_top_rated(api_key)
    return [x['id'] for x in data['results']]


In [ ]:
## returns full details when given a list of movie IDs
def get_movies(api_key,ids):
    data = []
    for i in ids:
        url = "https://api.themoviedb.org/3/movie/{}?api_key={}&language=en-US".format(i,api_key)
        time.sleep(.5)
        data.append(requests.get(url).json())
    return data

In [ ]:
## returns movie data when given an API key and list of movie titles 

def search_movies(api_key, list_of_movies):
    data = []
    for movie in list_of_movies[0:40]:
        url = "https://api.themoviedb.org/3/search/movie?api_key={}&language=en-US&query={}".format(api_key,movie)
        data.extend(requests.get(url).json()['results'])
        time.sleep(1)
    #time.sleep(10)
    for movie in list_of_movies[40:]:
        url = "https://api.themoviedb.org/3/search/movie?api_key={}&language=en-US&query={}".format(api_key,movie)
        data.extend(requests.get(url).json()['results'])
        time.sleep(1)
    return data


In [ ]:
# def search_movies(api_key, list_of_movies):
#     data = []
#     for movie in list_of_movies[0:40]:
#         url = "https://api.themoviedb.org/3/search/movie?api_key={}&language=en-US&query={}".format(api_key,movie)
#         temp = requests.get(url).json()['results']
#         data.extend(temp[0])
#         time.sleep(1)
#     time.sleep(21)
#     for movie in list_of_movies[41:]:
#         url = "https://api.themoviedb.org/3/search/movie?api_key={}&language=en-US&query={}".format(api_key,movie)
#         temp = requests.get(url).json()['results']
#         data.extend(temp[0])
#         time.sleep(1)
#     return data


In [ ]:
## search Movie DB for the best movies on Hulu according to Vulture 

hulu_movies = search_movies(api_key, vulture.best_hulu)

In [ ]:
netflix_movies = search_movies(api_key, vulture.best_netflix)

In [ ]:
amazon_movies = search_movies(api_key, vulture.best_amazon)

In [ ]:
print(len(hulu_movies)+len(netflix_movies)+len(amazon_movies))
all_ids = {x['id'] for x in hulu_movies}|{x['id'] for x in netflix_movies}|{x['id'] for x in amazon_movies}
len(set((all_ids)))


In [ ]:
get_movies_test = get_movies(api_key,set([x['id'] for x in hulu_movies][0:41]))

In [ ]:
all_movie_data = get_movies(api_key,set(all_ids))


In [ ]:
df = pd.DataFrame(all_movie_data)
df.head()
df_rev_budget = df[['imdb_id','title','revenue','budget','tagline','runtime','genres']]
df_rev_budget = df_rev_budget[df_rev_budget['revenue']>0]
df_rev_budget['gross_profit']=df['revenue']-df['budget']
df_rev_budget.sort_values('gross_profit',ascending=False,inplace=True)
df_rev_budget

In [ ]:
df_rev_budget['genres']=df_rev_budget['genres'].map(lambda x: [y['id'] for y in x])
df_rev_budget.head()

In [ ]:
df_rev_budget.head()

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
for column in ['status']:
    print(df[column].value_counts())